# Comet.ml REST API

Comet.ml has an extensive interface to all of your data via a [REST API](https://en.wikipedia.org/wiki/Representational_state_transfer) through [Comet.ml endpoints](https://www.comet.ml/docs/rest-api/endpoints/). Now, you can access this information easily through the Comet.ml Python SDK.

## Setup

To run the following experiments, you'll need to set your COMET_API_KEY and COMET_REST_API_KEY. The easiest way to to this is to set the values in a cell like this:

```python
%%writefile .env
COMET_API_KEY="..."
COMET_REST_API_KEY=".."
```

You can get your COMET_API_KEY under your quickstart link:

https://www.comet.ml/YOURID/quickstart

You can get your COMET_REST_API_KEY under settings

https://www.comet.ml/dsblank/settings/account

## Quick Overview

To access the REST API through the comet.ml SDK, you will need to make an API() instance. First, we import the API class:

In [1]:
from comet_ml import API

ConfigurationError: Invalid variable name "comet.api_key" in env file (line 9)

and creat the instance:

In [7]:
api = API()

In [8]:
api.workspaces

['dsblank']

In [9]:
api.workspaces["cometpublic"]

['fasttext', 'comet-notebooks', 'parameter-space-exploration', 'home-credit']

In [10]:
api.workspaces["cometpublic"]["comet-notebooks"]

['7092a5e4c362453fb0b3f06785a1d30c', 'd21f94a1c71841d2961da1e6ddb5ab20']

In [11]:
api.workspaces["cometpublic"]["comet-notebooks"]['d21f94a1c71841d2961da1e6ddb5ab20']

<Experiment 'cometpublic/comet-notebooks/d21f94a1c71841d2961da1e6ddb5ab20'>

In [19]:
exp = api.workspaces["cometpublic"]["comet-notebooks"]['d21f94a1c71841d2961da1e6ddb5ab20']

In [20]:
exp

<Experiment 'cometpublic/comet-notebooks/d21f94a1c71841d2961da1e6ddb5ab20'>

In [21]:
exp.other

[{'name': 'Name',
  'valueMax': 'example 001',
  'valueMin': 'example 001',
  'valueCurrent': 'example 001',
  'timestampMax': 1543866627717,
  'timestampMin': 1543866627717,
  'timestampCurrent': 1543866627717}]

In [22]:
api.workspaces["cometpublic"]["comet-notebooks"]['example 001']

<Experiment 'cometpublic/comet-notebooks/example 001'>

## Workspaces

In [23]:
api.workspaces

['dsblank']

In [24]:
for workspace in api.workspaces:
    print(workspace)

dsblank


## Projects

In [25]:
api.workspaces["cometpublic"]

['fasttext', 'comet-notebooks', 'parameter-space-exploration', 'home-credit']

In [26]:
project = api.workspaces["cometpublic"]["comet-notebooks"]

In [28]:
project

['7092a5e4c362453fb0b3f06785a1d30c', 'd21f94a1c71841d2961da1e6ddb5ab20']

In [27]:
project.data

{'project_id': '63776506878949eb9ac225a2a24b87f6',
 'user_name': 'cometpublic',
 'project_name': 'comet-notebooks',
 'project_desc': '',
 'team_id': 'cometpublic-default',
 'is_owner': False,
 'is_public': True,
 'is_shared': False,
 'num_of_experiments': 2,
 'last_updated': 1543867086997,
 'team_name': 'cometpublic',
 'experiments': ['7092a5e4c362453fb0b3f06785a1d30c',
  'd21f94a1c71841d2961da1e6ddb5ab20']}

## Experiments

In [29]:
api.workspaces["cometpublic"]["comet-notebooks"]

['7092a5e4c362453fb0b3f06785a1d30c', 'd21f94a1c71841d2961da1e6ddb5ab20']

In [36]:
exp = api.workspaces["cometpublic"]["comet-notebooks"]['d21f94a1c71841d2961da1e6ddb5ab20']

In [37]:
exp.data

{'code_sha': '66e8551f',
 'file_name': 'Jupyter interactive',
 'file_path': 'Jupyter interactive',
 'duration_millis': 1043394000,
 'start_server_timestamp': 1542823692825,
 'end_server_timestamp': 1543867086997,
 'has_images': False,
 'experiment_key': 'd21f94a1c71841d2961da1e6ddb5ab20',
 'metrics': ['train_accuracy',
  'train_curr_epoch',
  'train_loss',
  'train_val_loss'],
 'other': ['Name'],
 'parameters': ['f']}

In [35]:
exp.other

[{'name': 'trainable_params',
  'valueMax': '134794',
  'valueMin': '134794',
  'valueCurrent': '134794',
  'timestampMax': 1542824535416,
  'timestampMin': 1542824535416,
  'timestampCurrent': 1542824535416}]

In [38]:
exp.display()